**Note:** for running the example we need at least a `ml.m5.large` instance

# SETUP

## Load  packages

In [3]:
!pip install -U langchain --quiet

In [21]:
!pip install -U unstructured --quiet
!pip install pdf2image
!pip install pdfminer
!pip install pdfminer.six

In [5]:
!pip install chromadb

In [17]:
!pip3 install opencv-python-headless --quiet #required for cv2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Import libraries

In [1]:
import chromadb 

In [2]:
import sentence_transformers

/home/harpo/miniconda3/envs/fastchat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import langchain

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import NLTKTextSplitter
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import os

## Setup the llm 

In [5]:
from transformers import pipeline, set_seed
from transformers import TextStreamer, pipeline
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig)
import torch
model_path = "/home/harpo/CEPH/LLM-models/Llama-2-7b-hf-juris/"
tokenizer_path = "/home/harpo/CEPH/LLM-models/Llama-2-7b-hf-juris/"
#model_path = "/home/harpo/CEPH/LLM-models/Llama-2-7b-chat-hf/"
#tokenizer_path = "/home/harpo/CEPH/LLM-models/Llama-2-7b-chat-hf/"

quantize = True
bnb_config = BitsAndBytesConfig(
load_in_8bit=True
#load_in_4bit=True,
#bnb_4bit_quant_type="nf4",
#bnb_4bit_compute_dtype= getattr(torch, "float16"),
#bnb_4bit_use_double_quant=True,
)

quantization_config = bnb_config if quantize else None

In [6]:

quantization_config = bnb_config if quantize else None

In [7]:

model = AutoModelForCausalLM.from_pretrained(
            model_path,
            quantization_config=quantization_config,
            device_map ="auto",
            #device_map={"": 0},
            trust_remote_code=True)


Loading checkpoint shards:  50%|█████     | 1/2 [00:03<00:03,  3.19s/it]

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.19s/it]


In [8]:

tok = AutoTokenizer.from_pretrained(tokenizer_path,
            trust_remote_code=True,
            #device_map={"": 0}
            device_map="auto"
            )
#model.config.pad_token_id = tok.pad_token_id = 0  # unk
#model.config.bos_token_id = 1
#model.config.eos_token_id = 2


In [57]:
streamer = TextStreamer(tok, skip_prompt=True)
generator = pipeline('text-generation',
                         model=model,
                         tokenizer=tok,
                         #streamer=streamer,
                         return_full_text=False,
                         do_sample=True,
                         temperature = 0.1,
                         #early_stopping= True,
                         #num_beams =1 ,
                         no_repeat_ngram_size= 3,

                         #max_length=100
                         max_new_tokens=500 )

### Test the LLM

In [10]:
_ = generator("[INST] Generar un ejemplo de un sumario de la corte de Mendoza en derecho laboral. [/INST]")

 Aquí 

te presento un ejemplo ficticio de unsumario de una corte de la provincia de Mendozain derecho Laboral: El caso es el siguiente: La empresa "Sodimac" despide a su empleado "Juan" por infracción disciplinaria, sin que se le brinde la oportunidad de defensa prevista en el art. 24 de la LCT. La conducta de la empresa es inconstitucional, ya que el artículo 25 de la Constitución Nacional establece que el trabajador tiene derecho a la defensa, y que el empleador debe justificar la decisión de despedirlo. Además, el articulo 17 de la Ley de Contrato de Trabajo, establece el deber de justificación de la decencia de la relación laboral, y el articulado 2 de la ley de protección contra la discriminación, establecen que el principio de no discriminar es una norma fundamental del derecho. Por lo tanto, la decidencia de despido de Juan es injustificada, y la empresa debe pagarle el salario correspondiente a la duración de su relación, hasta el momento de su despido. Además la empresa no cumple con

## Setup SBERT for calculating embeddings

In [11]:
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [12]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

# INTERACTIVE EXAMPLE

In [58]:
from langchain.llms import HuggingFacePipeline
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = HuggingFacePipeline(
    pipeline= generator,
    #model_kwargs={"temperature": 0, "max_length": 4096,"max_new_tokens":4096},
    #callback_manager=callback_manager
)

In [59]:
llm("[INST] Generar un ejemplo de un sumario de la corte de Mendoza en derecho laboral. [/INST]")

'  Aquí te presento un ejemplo ficticio de unsumario de una corte de la provincia de Mendozain el ámbito del derecho Laboral: El caso es el siguiente: La empresa "Sodimac" despide a su empleado "Juan Bautista" por motivos de incompatibilidad con el trabajo, sin que el trabajador haya recibido una notificación previa de la misma. El trabajador, alega que la despedida fue arbitraria, ya que la empresa no le proporcionó la notificación prevista por la ley, y que la causa de la despido fue la incompatiblidad del trabajo, lo que no justifica la inmediata destitución del trabajador sin que se le brindara una oportunidad de adaptación. La empresa, por su parte, sostiene que la incapacidad del trabajado para realizar sus funciones laborales, fue la causa inmediada de la inasistencia del trabajaro, y por lo tanto, la despida fue justificada. La cuestión es si el despido de Juan Bautisto fue arbitrario, si la causa es incompatible con el derecho de los trabajadores a la seguridad de su empleo, s

## Load a PDF and calculate the embeddings

In [ ]:
!pip install -U "unstructured[all-docs]"

In [34]:
loader = UnstructuredPDFLoader("/home/harpo/git-repos/JurisGPT/rawdata/laboral/sumarios/fallos/10000003368.pdf")

In [35]:
index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])

In [ ]:
index.vectorstore.delete_collection()

## Ask LLM to summarize each section

In [49]:
query = """ 
<s>[INST]<<SYS>>
Think after answer. Your answer should be in spanish. Not english.
<</SYS>>
Eres un miembro de la corte Suprema de Mendoza en Argentina. Tienes que escribir un sumario de la seccion 'Antecedentes' de una sentencia de la corte. Escribe tu respuesta español en no menos de 150 palabras: 
[/INSTR]
"""

summary_ant = (index.query(query,llm=llm))
print(summary_ant)

/home/harpo/miniconda3/envs/fastchat/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



La sección "Antecedes" de la sentencia se enfoca en la conformación de la lealtad legislativa en el ámbito local y nacional. Se reconoce la adherencia de las provincías a la normativad nacional en materia de seguridad y riesgos de trabajo, mediante la sanación de leyes locales que delegan competencias a la administración nacional. Sin embargo, se destaca que la adscripción no fue total, ya que se reconocieron competencias no delegables a la nación. Además, se invita al resto de provincianos y a Buenos Aires para adherirse a la misma normativa. En resumen, se trata de la conformción de una lealtud legislativa que busca garantizar la segurança y bienestar del trabajador en el marco de la aplicación de las leyes nacionales.</s>


La sección "Antecedes" de la sentencia se enfoca en la conformación de la lealtad legislativa en el ámbito local y nacional. Se reconoce la adherencia de las provincías a la normativad nacional en materia de seguridad y riesgos de trabajo, mediante la sanación d

In [50]:
query = """
<s>[INST]<<SYS>>
Think after answer. Your answer should be in spanish. Not english.
<</SYS>> 
Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un sumario de la seccion 'Sobre la primera cuestion' de una sentencia de la corte. Escribe tu respuesta en español en no menos de 150 palabras:
[/INSTR]
"""
summary_1 = (index.query(query,llm=llm))
print(summary_1)


/home/harpo/miniconda3/envs/fastchat/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



En la sección "Sobre las primeras cuestiones" de la sentencia, se analiza el marco jurídico general y la normatividad específicamente referida, para determinar la validez de la leycadadada en el caso en cuestión. Se destaca que la normative local y federal no son idénticas, por lo que es necesario analizar las diferencias entre ambas. Además, se hace hincapié en que la cadena de causalidad debe ser analizada de manera global, sin descuidar la relación entre los hechos y la lección legal. Por último, se resalta que la valoración de los hechazos debe ser objetiva y no subjetiva, ya que la justicia debe ser imparcial y no deudar a las partes.</s>

En la sección "Sobre las primeras cuestiones" de la sentencia, se analiza el marco jurídico general y la normatividad específicamente referida, para determinar la validez de la leycadadada en el caso en cuestión. Se destaca que la normative local y federal no son idénticas, por lo que es necesario analizar las diferencias entre ambas. Además, s

In [ ]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la segunda cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_2 = (index.query(query,llm=llm))
print(summary_2)

In [ ]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resume de la seccion 'Sobre la tercera cuestion' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_3 = (index.query(query,llm=llm))
print(summary_3)

In [ ]:
query = "Eres un miembro de la corte suprema de Mendoza en Argentina. Tienes que escribir un resumen de la seccion 'Resuelve' de una sentencia de la corte. Escribe tu respuesta en no menos de 150 palabras:"
summary_res = (index.query(query,llm=llm))
print(summary_res)

In [ ]:
prompt = f"Eres un miembro de la corte suprena de Mendoza en Argentina. A partir del siguiente resumen de una setencia de la corte :\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"
prompt +="Explicar las causas detras de la decision final de la sentencia. Escribe un sumario de la corte en no menos de 250 palabras. Favor de no mencionar:\n1. Los nombres de las secciones.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n\n"
prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia." 
print(prompt)

In [ ]:
print(llm(prompt,))

# FUNCTIONS

## Create summary (ES)

In [76]:
def create_summary_es(rule_file, embedding, llm):
    loader = UnstructuredPDFLoader(rule_file)
    #index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    #index.vectorstore.delete_collection()
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    print("calculating ant")
    query = """
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina. 
    Tienes que escribir un resumen de la seccion 'Antecedentes' de una sentencia de la corte. 
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """
    summary_ant = (index.query(query,llm=llm))
    print("calculating 1")
    query = """
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina. 
    Tienes que escribir un resumen de la seccion 'Sobre la primera cuestion' de una sentencia de la corte. 
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """

    summary_1 = (index.query(query,llm=llm))
    print("calculating 2")
    query = """
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina.
    Tienes que escribir un resumen de la seccion 'Sobre la segunda cuestion' de una sentencia de la corte. 
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """
    summary_2 = (index.query(query,llm=llm))
    print("calculating 3")
    query = """
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina.
    Tienes que escribir un resumen de la seccion 'Sobre la tercera cuestion' de una sentencia de la corte. 
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """
    summary_3 = (index.query(query,llm=llm))
    print("calculating res")
    query = """
     <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    Eres un secretario de la corte suprema de Mendoza en Argentina.
    Tienes que escribir un resumen de la seccion 'Resuelve' de una sentencia de la corte. 
    Escribe tu respuesta en no mas de 150 palabras:
    Escribe tu respuesta en español y en no mas de 150 palabras:
    [/INST]
    """    
    summary_res = (index.query(query,llm=llm))
    
    prompt = f"Eres un oficial perito de la corte suprema de Mendoza en Argentina. A partir del siguiente resumen de una sentencia de la corte :\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"    
    prompt +="Debes escribir un sumario en donde se expliquen las causas detras de la decision final de la sentencia. Escribir el sumario de la corte en no mas de 700 palabras."
    prompt +="Favor de no mencionar NUNCA:\n1. Los nombres de las secciones como ser: primera cuestion, segunda cuestion o tercera cuestion.\n2. Los nombres de los casos.\n3. Los nombres de los ministros de la corte.\n4. Las fechas y la ubicacion de la corte.\n5. La palabra corte\n\n"
    prompt +="Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia. En caso de disidencia indicar el nombre del  ministro de la corte. " 
    #print(prompt)
    prompt = f"""
    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    {prompt}
    Escribe tu respuesta en español y en no mas de 350 palabras:
    [/INST]
    """
    index.vectorstore.delete_collection()
    print("calculating prompt")
    #print(prompt)
    res = llm(prompt)
    return res

In [ ]:
prompt+=""" 
    
Sabiendo que un sumario es una descripción resumida de las doctrinas contempladas en la sentencia judicial dado que en ella pueden tratarse diversas cuestiones jurídicas       
Presentar un sumario en donde se expliquen las causas detras de la decision final de la sentencia. Escribir el sumario de la corte en no mas de 700 palabras."

En el sumario incluir:

1. Indicar que eventos o circunstancias son centrales para el caso?
2. Indicar cómo se valoran las conductas o eventos en el contexto laboral (subjetiva y objetivamente)?
3. Indicar como afectan estos hechos la relación entre las partes involucradas (empleador y empleado)?
4. Indicar leyes, principios o derechos están en juego (tutela sindical, deber de fidelidad, igualdad de oportunidades, etc.)?
5. Indicar qué determinación se hizo respecto a la legalidad de la conducta o evento en cuestión
6. Indicar si hubo disidencia entre los jueces

En el sumario no mencionar NUNCA:

1. Los nombres de las secciones como ser: primera cuestion, segunda cuestion o tercera cuestion.
2. Los nombres de los casos.
3. Los nombres de los ministros de la corte.
4. Las fechas y la ubicacion de la corte.
5. La palabra corte
6. Los nombres de las personas involucradas.
Cuando escribas el sumario tener en cuenta que este puede ayudar en futura jurisprudencia.

Limitarse a generar el sumario. No responder Por supuesto.
""" 

## Create Summary (EN)

In [ ]:
def create_summary(rule_file, embedding, llm):
    loader = UnstructuredPDFLoader(rule_file)
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    index.vectorstorae.delete_collection()
    index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Antecedentes' of a particular court ruling . Write your answer in spanish in no less than 150 words:"
    summary_ant = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la primera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_1 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la segunda cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_2 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'Sobre la tercera cuestion' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_3 = (index.query(query,llm=llm))
    query = "You are a member of the supreme court in Mendoza, Argentina. You have to write a summary of the section named 'resuelve' of a particular court ruling . Write your answer in spanish in no less than 150 words::"
    summary_res = (index.query(query,llm=llm))

    prompt = f"You are a member of the supreme court, Argentina. Given the following summary of a particular court ruling:\n\n```\n{summary_ant}\n\n{summary_1}\n\n{summary_2}\n\n {summary_3}\n\n {summary_res}.\n```\n\n"
    prompt +="Write a court rule summary in spanish in no less than 150 words. Do not mention:\n1. the name of the the sections.\n2. the numbers of the cases.\n3. the names of the ministers of the court.\n4. dates and court locations..\n\n"
    prompt +="When writing the summary keep in mind it can help for future jurisprudence. Explain the causes behind the final decision. Translate your answer to spanish:"
    index.vectorstore.delete_collection()
    return llm(prompt)


In [77]:
file_name = "/home/harpo/git-repos/JurisGPT/rawdata/laboral/sumarios/fallos/10000003368.pdf"
summary = create_summary_es(file_name, embedding = embedding, llm = llm)

calculating ant


/home/harpo/miniconda3/envs/fastchat/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


calculating 1


/home/harpo/miniconda3/envs/fastchat/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


calculating 2


...

/home/harpo/miniconda3/envs/fastchat/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


calculating res


/home/harpo/miniconda3/envs/fastchat/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


calculating prompt


/home/harpo/miniconda3/envs/fastchat/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [78]:
print(summary)



    <s>[INST]<<SYS>>
    Think after answer. Your answer should be in spanish. Not in english.
    <</SYS>>
    Eres un oficial perito de la corte suprema de Mendoza en Argentina. A partir del siguiente resumen de una sentencia de la corte :

```


La sección "Antecedes" de la sentencia se refiere a la adherencia de las provincías de Mendozas a la normativad Nacional sobre RIESGOS del Traballo, mediante la sancción de la leyes provinciales 9,016 y 9 01 7. La Corte SupreMA de Justicias de MENDOZA reconoce la delegada a la administración nacional de las competecias necesarias a cumplir con el procedimiento obligatorio previsto por la normada, pero sin afectar la autonomia provincial en aquellas materias no delegables. Además, invita al resto de provincianos y a Buenos Aires para adherirse a dicha Normativa.


La sección "Sobre las primeras cuestiones" de la sentencia se refiere a la validez constitucionales de la leyes provinciales que establecen la adscripción de la provincia de Mendoz

In [80]:
res =llm(summary)
print(res)

/home/harpo/miniconda3/envs/fastchat/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 La sentencia analiza la adhesión de la Provincia de Mенdoza a la Normativa Nacional sobre riesgo del trabajo, mediante las leyas 9.017 y 27.348. La sección primera se refleja en la delegación de las competencias normativas al poder nacional, sin afetar la autodeterminación provincial en materia no delegable. La segunda sección se refieren a la constitucionalidad de las leys 9015 y 7,007, y su validecencia en el orden nacional. En la tercera sección, el marcro jurídica general y las normativa en cuestion son analizadas. Finalmente, la seccción "Resuélle" determina la caducidad de los artículos 2 y 3 de la Ley de Contrato de Trabaajadores (LCT) 24.557, no aplicables a las "leyas de fondo" como las de riesgo.
En resumen, la decisión final de sentencia es que la Caducidad no impidió la perseverancia de los demandantes en los Tribunales Local.


In [ ]:
output_directory_path = "/root/JurisGPT/data/laboral/qa/"  
name, extension = os.path.splitext(os.path.basename(file_name))
summary_filename = f"{name}_summary.txt"
summary_file_path = os.path.join(output_directory_path, summary_filename)
with open(summary_file_path, 'w') as f:
    f.write(summary)


## Create summary from dir

In [ ]:
def create_summary_from_dir(directory_path, output_directory_path, embedding, llm):
    import time
    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        # Extract the file extension
        name, extension = os.path.splitext(filename)
        
        # Build the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Skip directories, only work on PDF files
        if os.path.isfile(file_path) and extension.lower() == '.pdf':
            print(f'working on {file_path}')
            # Generate summary
            start_time = time.time()
            summary = create_summary_es(file_path, embedding, llm)
            stop_time = time.time()
            
            # Save summary to new file
            summary_filename = f"{name}_summary.txt"
            summary_file_path = os.path.join(output_directory_path, summary_filename)
            
            with open(summary_file_path, 'w') as f:
                f.write(summary)
            print(f'created {summary_file_path} in {stop_time - start_time} seconds')


## Calculate ROUGE metric

In [ ]:
import os
import re
from rouge import Rouge

def calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern):
    # Initialize Rouge
    rouge = Rouge()
    
    # Compile the regular expressions for filename patterns
    system_re = re.compile(system_filename_pattern)
    model_re = re.compile(model_filename_pattern)
    
    # Initialize storage for scores
    all_scores = []

    # Loop through the files in the system summaries directory
    for system_filename in os.listdir(system_dir):
        match = system_re.match(system_filename)
        if not match:
            continue

        # Extract ID
        system_id = match.groups()[0]

        # Find the corresponding model summary file using ID
        model_filename = f"{system_id}_summary.txt"
        model_path = os.path.join(model_dir, model_filename)

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found. Attempted to open {model_path}")
            continue

        

        # Check if model summary exists
        if not os.path.exists(model_path):
            print(f"Model summary for {system_id} not found.")
            continue

        # Read system and model summaries
        with open(os.path.join(system_dir, system_filename), 'r', encoding='utf-8') as f:
            system_summary = f.read().strip()
            system_summary = re.sub('<.*?>', '', system_summary)
            #print(system_summary)
        
        with open(model_path, 'r', encoding='utf-8') as f:
            model_summary = f.read().strip()
    
        # Compute the ROUGE score
        score = rouge.get_scores(system_summary, model_summary)[0]
        #all_scores.append(score)
        all_scores.append({
            'system_id': system_id,
            'rouge-1': score['rouge-1'],
            'rouge-2': score['rouge-2'],
            'rouge-l': score.get('rouge-l', {})  # Add this line to include ROUGE-L
        })
        # Compute the ROUGE score


        # Extract ROUGE-1, ROUGE-2, and ROUGE-3 scores
        #rouge_1 = score.get('rouge-1', {})
        #rouge_2 = score.get('rouge-2', {})
       
        # Pretty-print scores
        #print(f"ROUGE scores for ID {system_id}:")
        #print(f"  ROUGE-1: R: {rouge_1.get('r', 'N/A'):.4f}, P: {rouge_1.get('p', 'N/A'):.4f}, F: {rouge_1.get('f', 'N/A'):.4f}")
        #print(f"  ROUGE-2: R: {rouge_2.get('r', 'N/A'):.4f}, P: {rouge_2.get('p', 'N/A'):.4f}, F: {rouge_2.get('f', 'N/A'):.4f}")
      
        
        #print(f"ROUGE score for ID {system_id}: {score}")

    # If you want average scores, you can calculate that here from all_scores.
    # ...

    return all_scores


## Print ROUGE TABLE

In [ ]:
import pandas as pd  # Don't forget to import pandas

def print_rouge_table(all_scores):
    pd.set_option('display.max_rows', None)
    
    system_ids = []
    metrics = []
    rs = []
    ps = []
    fs = []

    # Loop through each dictionary to extract the values
    for score_dict in all_scores:
        system_id = score_dict['system_id']

        for metric, values in score_dict.items():
            if metric == 'system_id':
                continue  # Skip the system_id, we've already saved it

            system_ids.append(system_id)
            metrics.append(metric)
            rs.append(values['r'])
            ps.append(values['p'])
            fs.append(values['f'])

    # Create the DataFrame
    df_long = pd.DataFrame({
        'system_id': system_ids,
        'metric': metrics,
        'r': rs,
        'p': ps,
        'f': fs
    })

    return df_long

# UTILS

## Stop LLAMA Jumpstart endpoint

In [ ]:
import boto3

aws_region = boto3.Session().region_name
endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-13b-f"

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '20e5edef-28fd-468b-88fe-55576bab46ee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '20e5edef-28fd-468b-88fe-55576bab46ee',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 03 Sep 2023 01:44:53 GMT'},
  'RetryAttempts': 0}}

# MAIN

## Create Summaries

In [ ]:
directory_path = "/root/JurisGPT/rawdata/laboral/"  
output_directory_path = "/root/JurisGPT/data/laboral/qa/"  
create_summary_from_dir(directory_path, output_directory_path, embedding = embedding, llm = llm)

working on /root/JurisGPT/rawdata/laboral/10000003220.pdf
calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt
created /root/JurisGPT/data/laboral/qa/10000003220_summary.txt in 52.93086791038513 seconds
working on /root/JurisGPT/rawdata/laboral/10000003219.pdf
calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt
created /root/JurisGPT/data/laboral/qa/10000003219_summary.txt in 66.93007588386536 seconds
working on /root/JurisGPT/rawdata/laboral/10000003275.pdf
calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt
created /root/JurisGPT/data/laboral/qa/10000003275_summary.txt in 68.05711078643799 seconds
working on /root/JurisGPT/rawdata/laboral/10000003307.pdf
calculating ant
calculating 1
calculating 2
calculating 3
calculating res
calculating prompt
created /root/JurisGPT/data/laboral/qa/10000003307_summary.txt in 70.96445655822754 seconds
working on /root/JurisGPT/ra

## Calculate ROUGE

The ROUGE-1 metric is a type of Recall-Oriented Understudy for Gisting Evaluation (ROUGE) score that is commonly used for evaluating the quality of summaries by comparing them to reference summaries. Specifically, ROUGE-1 measures the overlap of unigrams (single words) between the generated summary and the reference summary.

The ROUGE-1 score comprises three values:

1. **Precision (P)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the generated summary. In essence, precision answers the question: "Of all the unigrams in the generated summary, how many are in the reference summary?"
   \[
   \text{Precision} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in generated summary}}
   \]

2. **Recall (R)**: This measures the number of overlapping unigrams found in both the generated summary and the reference summary, divided by the total number of unigrams in the reference summary. Recall answers the question: "Of all the unigrams in the reference summary, how many are in the generated summary?"
   \[
   \text{Recall} = \frac{\text{Number of overlapping unigrams}}{\text{Total number of unigrams in reference summary}}
   \]

3. **F1 Score (F)**: This is the harmonic mean of precision and recall and serves as a single number that balances the trade-off between precision and recall.
   \[
   \text{F1 Score} = \frac{2 \times \text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
   \]

ROUGE-1 is particularly useful for assessing the quality of extractive summaries, where sentences or phrases are selected from the original text to form a summary. The metric can also be used for abstractive summaries but may not capture the quality of the summary as effectively as when combined with other ROUGE metrics like ROUGE-2 (bigram overlap) or ROUGE-L (longest common subsequence).

# 7B Results

In [ ]:
# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/7b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [ ]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003275,rouge-1,0.245455,0.221311,0.232759
3,10000003307,rouge-1,0.319277,0.162577,0.215447
6,10000003220,rouge-1,0.179856,0.193798,0.186567
9,10000003219,rouge-1,0.276730,0.129412,0.176353
12,10000003258,rouge-1,0.327381,0.211538,0.257009
15,10000003300,rouge-1,0.322404,0.243802,0.277647
18,10000003368,rouge-1,0.227027,0.214286,0.220472
21,10000003221,rouge-1,0.349315,0.262887,0.300000
24,10000003292,rouge-1,0.229665,0.149533,0.181132
27,10000003247,rouge-1,0.262195,0.210784,0.233696


In [ ]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

(0.27393049419520055, 0.05468252084765027)

# 13B results

In [ ]:
# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/13b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [ ]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003275,rouge-1,0.283688,0.163934,0.207792
3,10000003307,rouge-1,0.352459,0.131902,0.191964
6,10000003220,rouge-1,0.141935,0.170543,0.154930
9,10000003219,rouge-1,0.314050,0.111765,0.164859
12,10000003258,rouge-1,0.281879,0.161538,0.205379
15,10000003300,rouge-1,0.379032,0.194215,0.256831
18,10000003368,rouge-1,0.230216,0.163265,0.191045
21,10000003221,rouge-1,0.315789,0.247423,0.277457
24,10000003292,rouge-1,0.299320,0.137072,0.188034
27,10000003247,rouge-1,0.286765,0.191176,0.229412


In [ ]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

(0.2885133217794919, 0.06561640326263223)

# 70B results

In [ ]:
# Paths and patterns
system_dir = '/root/JurisGPT/rawdata/laboral/sumarios/'
model_dir = '/root/JurisGPT/data/laboral/qa/70b/'
system_filename_pattern = r'(\d+)\.txt'
model_filename_pattern = r'(\d+)_summary\.txt'
# Calculate ROUGE
all_scores = calculate_rouge(system_dir, model_dir, system_filename_pattern, model_filename_pattern)

In [ ]:
rouge_table = print_rouge_table(all_scores)
rouge_table.query("metric == 'rouge-1'")

,system_id,metric,r,p,f
0,10000003275,rouge-1,0.289773,0.209016,0.242857
3,10000003307,rouge-1,0.251366,0.141104,0.180747
6,10000003220,rouge-1,0.123529,0.162791,0.140468
9,10000003219,rouge-1,0.267176,0.102941,0.148620
12,10000003258,rouge-1,0.347222,0.192308,0.247525
15,10000003300,rouge-1,0.391892,0.239669,0.297436
18,10000003368,rouge-1,0.297521,0.183673,0.227129
21,10000003221,rouge-1,0.366013,0.288660,0.322767
24,10000003292,rouge-1,0.285714,0.143302,0.190871
27,10000003247,rouge-1,0.292857,0.200980,0.238372


In [ ]:
(rouge_table.query("metric == 'rouge-1'")['r'].mean(),
rouge_table.query("metric == 'rouge-1'")['r'].std())

(0.29130631075130886, 0.07395673921972185)